In [1]:
# https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html
# https://medium.com/datadriveninvestor/choosing-the-best-algorithm-for-your-classification-model-7c632c78f38f
# https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html
# https://www.dataquest.io/blog/sci-kit-learn-tutorial/
# https://datascience.stackexchange.com/questions/33256/how-to-apply-machine-learning-model-to-new-dataset
# https://speakerdeck.com/datasciencela/tianqi-chen-xgboost-overview-and-latest-news-la-meetup-talk?slide=33

In [2]:
from fastai.tabular import *
import os
import pandas as pd 
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB
from sklearn import svm, tree
import xgboost
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.kernel_approximation import RBFSampler


from sklearn import model_selection
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt

from xgboost import plot_importance
from xgboost import plot_tree
import xgboost as xgb

import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss


In [3]:
path = Path('/Users/bbabu/fastai/DataSamples')

data = pd.read_csv(path/'Output-5v5.csv', sep=',', header=0)
df_test = pd.read_csv(path/'Output-5v5-Test.csv', sep=',', header=0)
df_test_2 = pd.read_csv(path/'Output-5v5-Test.csv', sep=',', header=0)


data.drop(['Description', 'Platform', 'BusinessGroup', 'Country'], axis=1, inplace=True)
df_test.drop(['Description', 'Platform', 'BusinessGroup', 'Country'], axis=1, inplace=True)
df_test_2.drop(['Description', 'Platform', 'BusinessGroup', 'Country'], axis=1, inplace=True)




In [4]:
data = pd.read_csv(path/'Groups_with_Owners-V2.csv', sep=',', header=0)
df_test = pd.read_csv(path/'Groups_without_Owners-V2.csv', sep=',', header=0)
df_test_2 = pd.read_csv(path/'Groups_without_Owners-V2.csv', sep=',', header=0)

In [5]:
# data = pd.read_csv(path/'Groups_with_Owners-V1.csv', sep=';', header=0)
# df_test = pd.read_csv(path/'Groups_without_Owners-V1.csv', sep=';', header=0)
# df_test_2 = pd.read_csv(path/'Groups_without_Owners-V1.csv', sep=';', header=0)

In [6]:
data.drop(['Description', 'MemberDN', 'State'], axis=1, inplace=True)
df_test.drop(['Description', 'MemberDN', 'State', 'Owner'], axis=1, inplace=True)
df_test_2.drop(['Description', 'MemberDN', 'State', 'Owner'], axis=1, inplace=True)

In [7]:
# data.drop(['GroupCountry', 'GroupDomain', 'Domain'], axis=1, inplace=True)
# df_test.drop(['GroupCountry', 'GroupDomain', 'Domain'], axis=1, inplace=True)
# df_test_2.drop(['GroupCountry', 'GroupDomain', 'Domain'], axis=1, inplace=True)

In [8]:
data.drop(['GroupCountry'], axis=1, inplace=True)
df_test.drop(['GroupCountry'], axis=1, inplace=True)
df_test_2.drop(['GroupCountry'], axis=1, inplace=True)

In [9]:
data.drop_duplicates(keep='first',inplace=True) 
df_test.drop_duplicates(keep='first',inplace=True)
df_test_2.drop_duplicates(keep='first',inplace=True) 

In [10]:
# print(data.dtypes)
# print(df_test.dtypes)
# print(df_test_2.dtypes)

In [11]:
le = preprocessing.LabelEncoder()
categorical = list(data.select_dtypes(include=['object']).columns.values)
for cat in categorical:
    #print(cat)
    data[cat].fillna('NaN', inplace=True)
    data[cat] = le.fit_transform(data[cat].astype(str))


In [12]:
le = preprocessing.LabelEncoder()

categorical = list(df_test_2.select_dtypes(include=['object']).columns.values)
for cat in categorical:
    #print(cat)
    df_test_2[cat].fillna('NaN', inplace=True)
    df_test_2[cat] = le.fit_transform(df_test_2[cat].astype(str))


In [13]:
# data.set_index('GroupDN', drop=False, inplace=True)

In [14]:
#Create Dependent and Independent Datasets based on our Dependent #and Independent features
# X = data[['GroupDN', 'Member', 'Manager', 'BusinessGroup', 'Platform', 'Title', 'Department', 'Country', 'City', 'GroupCountry', 'GroupDomain', 'Domain']]
X = data[['GroupDN', 'Member', 'Manager', 'BusinessGroup', 'Platform', 'Title', 'Department', 'Country', 'City', 'GroupDomain', 'Domain']]
# X = data[['GroupDN', 'Member', 'Manager', 'BusinessGroup', 'Platform', 'Title', 'Department', 'Country', 'City']]
y = data['Owner']
#Split the Data into Training and Testing sets with test size as #33%
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, shuffle=False)
eval_set = [(X_train, y_train), (X_test, y_test)]

In [15]:
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

In [16]:
# data.reindex(columns=)

In [17]:
model = xgboost.XGBClassifier(learning_rate=0.0001, 
                              max_depth=50, 
                              n_estimators=50, 
#                               seed=14,
#                               eval_metric='error',
                              booster='gbtree', 
                              objective='binary:logistic',
                              base_score=0.5,
                              colsample_bylevel=1,
                              colsample_bynode=1, 
                              colsample_bytree=1, 
                              gamma=0.2,
                              max_delta_step=0, 
                              min_child_weight=1, 
                              missing=None, 
                              n_jobs=500,
                              nthread=1000, 
                              random_state=0,
                              reg_alpha=0, 
                              reg_lambda=1, 
                              scale_pos_weight=1, 
                              subsample=1, 
                              verbosity=3)
model.fit(X_train, y_train)

[15:47:39] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4384 extra nodes, 182 pruned nodes, max_depth=26
[15:47:40] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4364 extra nodes, 200 pruned nodes, max_depth=26
[15:47:41] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4378 extra nodes, 184 pruned nodes, max_depth=26
[15:47:42] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4374 extra nodes, 186 pruned nodes, max_depth=26
[15:47:43] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4328 extra nodes, 158 pruned nodes, max_depth=26
[15:47:44] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4318 extra nodes, 146 pruned nodes, max_depth=26
[15:47:45] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4226 extra nodes, 148 pruned nodes, max_depth=26
[15:47:46] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4166 extra nodes, 166 pruned nodes, max_depth=26
[15:47:48] INFO:

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.2,
              learning_rate=0.0001, max_delta_step=0, max_depth=50,
              min_child_weight=1, missing=None, n_estimators=50, n_jobs=500,
              nthread=1000, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=3)

In [18]:
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
logloss = log_loss(y_test, y_pred)

In [19]:
# Save model to disk
filename = path/'Final_Model-XGBoosterV6.sav'
pickle.dump(model, open(filename, 'wb'))

In [20]:
# filename = path/'Final_Model-XGBooster-Saved.sav'

In [21]:
# Load model from disk and use it to make new predictions
loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, y_test)
# print(result)

In [22]:
X_test = df_test_2
loaded_model = pickle.load(open(filename, 'rb'))
# X_test = sc.transform(X_test)
pred = loaded_model.predict(X_test)


[15:48:35] ======== Monitor:  ========
[15:48:35] ======== Monitor:  ========


In [23]:
# f_score = pd.Series(loaded_model.get_booster().get_fscore(),index=df_test_2.columns.values).sort_values(ascending=False)
# print(f_score)


In [24]:
# cover = pd.Series(loaded_model.get_booster().get_score(importance_type="cover"),index=df_test_2.columns.values).sort_values(ascending=False)
# print(cover)
# gain = pd.Series(loaded_model.get_booster().get_score(importance_type="gain"),index=df_test_2.columns.values).sort_values(ascending=False)
# print(gain)

In [25]:
# xgboost.plot_tree(loaded_model, rankdir='LR')
# # plt.show()
# fig = plt.gcf()
# fig.set_size_inches(400, 300)
# fig.savefig(path/'Tree.png')

In [26]:
# plot_importance(model)
# plt.show()

In [27]:
feature_imp = pd.Series(loaded_model.feature_importances_,index=df_test_2.columns.values).sort_values(ascending=False)

In [28]:
#df_test['Member'] = le.inverse_transform(df_test['Member'])
final_df = pd.DataFrame({'GroupDN': df_test['GroupDN'],'Member': df_test['Member'], 'Manager': df_test['Manager'], 'Owner': pred})

In [29]:
final_df.to_csv(path/'MLProcessed1-XGBoosterV6.csv', header=True, index=False)

In [30]:
grouped = final_df.groupby(['GroupDN'])

In [31]:
i = 0
for name, group in grouped:
    if group['Owner'].sum() == 0:
        i += 1
        

In [32]:
print("Feature importance table is as follows:")
print(feature_imp.to_string())
# print(pred)
print("Model accuracy is: %s"%(acc))
print("Model logloss is: %s"%(logloss))
print("Total number of predicted owners are: %s"%(pred.sum()))
print("Mean of owners are: %s"%(pred.mean()))
print("Total number of groups are: %s"%(len(grouped)))
print("Number of predicted groups with owners are: %s"%(len(grouped) - i))
# print("Accuracy score of the model is: %s"%(acc))

Feature importance table is as follows:
Manager          0.882230
BusinessGroup    0.028993
Domain           0.014244
Department       0.012571
Platform         0.010790
Member           0.009822
Country          0.009321
GroupDomain      0.009144
Title            0.008653
City             0.008462
GroupDN          0.005768
Model accuracy is: 0.9398477143346199
Model logloss is: 2.0775917968327193
Total number of predicted owners are: 88426
Mean of owners are: 0.09724847928963498
Total number of groups are: 48963
Number of predicted groups with owners are: 14315


In [33]:
# df_test_2.to_csv(path/'Encoded_without_Owners.csv', header=True, index=False)
# data.to_csv(path/'Encoded_with_Owners.csv', header=True, index=False)